<a href="https://cognitiveclass.ai"><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork/images/IDSN-logo.png" width="400"> </a>

# Convolutional Neural Networks with Keras

Estimated time needed **30** mins


In this lab, we will learn how to use the Keras library to build convolutional neural networks. We will also use the popular MNIST dataset and we will compare our results to using a conventional neural network.


## Objectives for this Notebook    
* How to use the Keras library to build convolutional neural networks
* Convolutional neural network with one set of convolutional and pooling layers
* Convolutional neural network with two sets of convolutional and pooling layers



## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
      
1. <a href="#Import-Keras-and-Packages">Import Keras and Packages</a>   
2. <a href="#Convolutional-Neural-Network-with-One-Set-of-Convolutional-and-Pooling-Layers">Convolutional Neural Network with One Set of Convolutional and Pooling Layers</a>  
3. <a href="#Convolutional-Neural-Network-with-Two-Sets-of-Convolutional-and-Pooling-Layers">Convolutional Neural Network with Two Sets of Convolutional and Pooling Layers</a>  

</font>
</div>


### Install the necessary libraries


Let's start by installing the keras libraries and the packages that we would need to build a neural network.


In [1]:
# All Libraries required for this lab are listed below. The libraries need to be installed on Skills Network Labs. 
# If you run this notebook on a different environment, e.g. your desktop, you may want to install these.
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0
!pip install matplotlib==3.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 MB 40.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 47.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 151.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5

#### Suppress the tensorflow warning messages
We use the following code to  suppress the warning messages due to use of CPU architechture for tensoflow.
You may want to **comment out** these lines if you are using the GPU architechture


In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Import Keras and Packages


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.utils import to_categorical

When working with convolutional neural networks in particular, we will need additional packages.


In [4]:
from keras.layers import Conv2D # to add convolutional layers
from keras.layers import MaxPooling2D # to add pooling layers
from keras.layers import Flatten # to flatten data for fully connected layers

## Convolutional Neural Network with One Set of Convolutional and Pooling Layers


In [5]:
# import data
from keras.datasets import mnist

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Let's normalize the pixel values to be between 0 and 1


In [6]:
X_train = X_train / 255 # normalize training data
X_test = X_test / 255 # normalize test data

Next, let's convert the target variable into binary categories


In [7]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1] # number of categories

Next, let's define a function that creates our model. Let's start with one set of convolutional and pooling layers.


In [8]:
def convolutional_model():
    
    # create model
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(16, (5, 5), strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

Finally, let's call the function to create the model, and then let's train it and evaluate it.


In [9]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
300/300 - 35s - 117ms/step - accuracy: 0.9252 - loss: 0.2794 - val_accuracy: 0.9732 - val_loss: 0.0867
Epoch 2/10
300/300 - 34s - 114ms/step - accuracy: 0.9776 - loss: 0.0777 - val_accuracy: 0.9807 - val_loss: 0.0601
Epoch 3/10
300/300 - 33s - 111ms/step - accuracy: 0.9837 - loss: 0.0549 - val_accuracy: 0.9842 - val_loss: 0.0470
Epoch 4/10
300/300 - 33s - 110ms/step - accuracy: 0.9866 - loss: 0.0438 - val_accuracy: 0.9852 - val_loss: 0.0415
Epoch 5/10
300/300 - 33s - 111ms/step - accuracy: 0.9888 - loss: 0.0369 - val_accuracy: 0.9882 - val_loss: 0.0349
Epoch 6/10
300/300 - 33s - 110ms/step - accuracy: 0.9910 - loss: 0.0293 - val_accuracy: 0.9880 - val_loss: 0.0361
Epoch 7/10
300/300 - 33s - 110ms/step - accuracy: 0.9927 - loss: 0.0241 - val_accuracy: 0.9886 - val_loss: 0.0342
Epoch 8/10
300/300 - 33s - 110ms/step - accuracy: 0.9937 - loss: 0.0205 - val_accuracy: 0.9889 - val_loss: 0.0333
Epoch 9/10
300/300 - 33s - 110ms/step - accuracy: 0.9949 - loss: 0.0169 - val_accuracy: 

------------------------------------------


## Convolutional Neural Network with Two Sets of Convolutional and Pooling Layers


Let's redefine our convolutional model so that it has two convolutional and pooling layers instead of just one layer of each.


In [10]:
def convolutional_model():
    
    # create model
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(8, (2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

Now, let's call the function to create our new convolutional neural network, and then let's train it and evaluate it.


In [11]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
300/300 - 37s - 124ms/step - accuracy: 0.8637 - loss: 0.4715 - val_accuracy: 0.9588 - val_loss: 0.1368
Epoch 2/10
300/300 - 36s - 119ms/step - accuracy: 0.9624 - loss: 0.1280 - val_accuracy: 0.9744 - val_loss: 0.0851
Epoch 3/10
300/300 - 35s - 118ms/step - accuracy: 0.9725 - loss: 0.0920 - val_accuracy: 0.9787 - val_loss: 0.0664
Epoch 4/10
300/300 - 35s - 118ms/step - accuracy: 0.9783 - loss: 0.0734 - val_accuracy: 0.9813 - val_loss: 0.0570
Epoch 5/10
300/300 - 36s - 119ms/step - accuracy: 0.9810 - loss: 0.0611 - val_accuracy: 0.9845 - val_loss: 0.0475
Epoch 6/10
300/300 - 36s - 119ms/step - accuracy: 0.9833 - loss: 0.0529 - val_accuracy: 0.9852 - val_loss: 0.0444
Epoch 7/10
300/300 - 36s - 119ms/step - accuracy: 0.9863 - loss: 0.0456 - val_accuracy: 0.9837 - val_loss: 0.0489
Epoch 8/10
300/300 - 35s - 118ms/step - accuracy: 0.9874 - loss: 0.0408 - val_accuracy: 0.9871 - val_loss: 0.0406
Epoch 9/10
300/300 - 35s - 118ms/step - accuracy: 0.9894 - loss: 0.0358 - val_accuracy: 

<h3>Practice Exercise 1</h3>


Let's see how batch size affects the time required and accuracy of the model training. 
For this, you can try to change batch_size to 1024 and check it's effect on accuracy


In [12]:
# Write your answer here
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))


Epoch 1/10
59/59 - 38s - 637ms/step - accuracy: 0.6391 - loss: 1.2943 - val_accuracy: 0.8843 - val_loss: 0.4074
Epoch 2/10
59/59 - 35s - 594ms/step - accuracy: 0.9062 - loss: 0.3188 - val_accuracy: 0.9355 - val_loss: 0.2252
Epoch 3/10
59/59 - 33s - 555ms/step - accuracy: 0.9393 - loss: 0.2035 - val_accuracy: 0.9568 - val_loss: 0.1511
Epoch 4/10
59/59 - 40s - 676ms/step - accuracy: 0.9554 - loss: 0.1487 - val_accuracy: 0.9654 - val_loss: 0.1154
Epoch 5/10
59/59 - 38s - 645ms/step - accuracy: 0.9643 - loss: 0.1216 - val_accuracy: 0.9703 - val_loss: 0.0957
Epoch 6/10
59/59 - 44s - 753ms/step - accuracy: 0.9693 - loss: 0.1026 - val_accuracy: 0.9735 - val_loss: 0.0871
Epoch 7/10
59/59 - 41s - 694ms/step - accuracy: 0.9729 - loss: 0.0895 - val_accuracy: 0.9759 - val_loss: 0.0742
Epoch 8/10
59/59 - 41s - 697ms/step - accuracy: 0.9758 - loss: 0.0806 - val_accuracy: 0.9788 - val_loss: 0.0678
Epoch 9/10
59/59 - 37s - 622ms/step - accuracy: 0.9777 - loss: 0.0737 - val_accuracy: 0.9805 - val_loss:

Double-click <b>here</b> for the solution.

<!-- Your answer is below:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))
-->


<h3>Practice Exercise 2</h3>


Now, let's see how number of epochs  affect the time required and accuracy of the model training. 
For this, you can keep the batch_size=1024 and epochs=25 and check it's effect on accuracy


In [13]:
# Write your answer here
model = convolutional_model()

# fit the model(change epochs to 25)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))


Epoch 1/25
59/59 - 35s - 598ms/step - accuracy: 0.6091 - loss: 1.4169 - val_accuracy: 0.8696 - val_loss: 0.4389
Epoch 2/25
59/59 - 33s - 557ms/step - accuracy: 0.9014 - loss: 0.3320 - val_accuracy: 0.9344 - val_loss: 0.2328
Epoch 3/25
59/59 - 32s - 548ms/step - accuracy: 0.9391 - loss: 0.2091 - val_accuracy: 0.9543 - val_loss: 0.1668
Epoch 4/25
59/59 - 32s - 551ms/step - accuracy: 0.9569 - loss: 0.1484 - val_accuracy: 0.9643 - val_loss: 0.1224
Epoch 5/25
59/59 - 33s - 556ms/step - accuracy: 0.9648 - loss: 0.1187 - val_accuracy: 0.9690 - val_loss: 0.1012
Epoch 6/25
59/59 - 33s - 554ms/step - accuracy: 0.9708 - loss: 0.1011 - val_accuracy: 0.9739 - val_loss: 0.0869
Epoch 7/25
59/59 - 33s - 559ms/step - accuracy: 0.9722 - loss: 0.0927 - val_accuracy: 0.9759 - val_loss: 0.0764
Epoch 8/25
59/59 - 33s - 558ms/step - accuracy: 0.9754 - loss: 0.0818 - val_accuracy: 0.9788 - val_loss: 0.0684
Epoch 9/25
59/59 - 33s - 557ms/step - accuracy: 0.9776 - loss: 0.0745 - val_accuracy: 0.9790 - val_loss:

Double-click <b>here</b> for the solution.

<!-- Your answer is below:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))


    -->


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!


<!--
## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2024-11-20  | 3.0  | Aman  |  Updated the library versions to current |
| 2020-09-21  | 2.0  | Srishti  |  Migrated Lab to Markdown and added to course repo in GitLab |



<hr>

## <h3 align="center"> © IBM Corporation. All rights reserved. <h3/>


## <h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>

